In [1]:
import pandas as pd 
import numpy as np

import ast
import nest_asyncio         # nest_asyncio는 파이썬의 비동기 코드 실행을 도와주는 패키지
nest_asyncio.apply()        # Jupyter 노트북과 같이 이벤트 루프가 이미 실행 중인 환경에서 비동기 코드를 여러 번 실행할 수 있도록 설정하는 데 사용
# 이거 안하면 무조건 에러남 

from datasets import Dataset 
from ragas import evaluate

In [2]:
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
)

metrics = [
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
]


In [3]:
def set_data(file):
    qa_data = pd.read_excel(file)
    for col in ['contexts']:
        qa_data[col] = qa_data[col].apply(ast.literal_eval)
        
    dataset = Dataset.from_pandas(qa_data)
    
    return qa_data, dataset

In [4]:
file = 'experiment_dataset\QA\setting1_retriever1_003.xlsx'

qa_data, dataset = set_data(file)

<>:1: SyntaxWarning: invalid escape sequence '\Q'
<>:1: SyntaxWarning: invalid escape sequence '\Q'
C:\Users\kim_h\AppData\Local\Temp\ipykernel_20880\1664861591.py:1: SyntaxWarning: invalid escape sequence '\Q'
  file = 'experiment_dataset\QA\setting1_retriever1_003.xlsx'


In [5]:
qa_data

,question,ground_truth,answer,contexts
0,"Flipkart는 언제 설립되었으며, 본사는 어디에 위치해 있나요?","Flipkart는 2007년에 설립되었으며, 본사는 벵갈루루에 위치해 있습니다.","Flipkart는 2007년 10월에 설립되었으며, 본사는 인도 벵갈루루에 위치해 ...",[Flipkart\n사이트 주소 httpswwwflipkartcom\n플립카트Fli...
1,인도 재무부의 Notification No. 09/2020-Customs (N.T....,2020년 2월 2일에 발표되었습니다.,인도 재무부의 Notification No. 09/2020-Customs (N.T....,[08\n2 인도의 무역구제규정 개정\n인도 재무부는 2020년 2월 2일자로 반덤...
2,카자흐스탄 내에서 한국 차량 브랜드의 시장 점유율은 얼마나 되나요?,카자흐스탄 내 신차 판매에서 한국 차량 브랜드 비중이 약 40%로 1위를 기록하고 ...,제공된 자료에는 카자흐스탄 내에서 한국 차량 브랜드의 시장 점유율에 대한 정보가 포...,[차이즈의 성공률을 높일 수 있을 것으로 예상되기 때문이다\n2 빠른 시승 및 유지...
3,한전KPS가 파키스탄에서 운영 대금을 회수하지 못한 이유는 무엇인가요?,한전KPS는 파키스탄 중앙은행(SBP)이 외화 부족 상황을 이유로 운영 대금 해외송...,제공된 자료에는 한전KPS가 파키스탄에서 운영 대금을 회수하지 못한 구체적인 이유에...,[여기서 발생한 비용을 어디까지 포함시킬 수 있을 것이냐는 얼마든지 논의 가능한\n...
4,최근 인도의 수입규제 급증 배경은 무엇인가요?,최근 인도의 수입규제 급증은 인도의 국내 산업보호를 위한 정치적 요구와 당국의 통상...,최근 인도의 수입규제 급증 배경은 여러 가지 요인에 기인합니다. 주요 배경은 다음과...,[상반기 수출은 2078억 달러로 전년 동기 대비 127 감소 수입은 3202억 달...
5,리비아 당국이 발주하는 인프라 구축 사업에 대해 어떤 점을 면밀히 검토해야 하나요?,"리비아 당국이 발주하는 각종 인프라 구축 내지 복구 사업에 있어 실질적 이행 능력,...",리비아 당국이 발주하는 인프라 구축 사업에 참여하려면 다음과 같은 점들을 면밀히 검...,[기관이 전력과 도로 그리고 상하수도 등 제반 인프라를 모두 제공하면서 분양 또는 ...
6,원산지 검증 시 어떤 위험이 있을 수 있나요?,"상대국 HS코드로만 원산지결정기준이 충족되는 경우, 원산지 검증 시 불충족 위험이 ...",원산지 검증 시 발생할 수 있는 위험은 여러 가지가 있습니다. 제공된 자료를 바탕으...,[조건이어서 어떤 영업 활동을 해서는 안된다 제품이나 서비스의 직접 판매는 물론이고...
7,2018년 한국의 수출액은 얼마였나요?,2018년 한국의 수출액은 6천억 달러였습니다.,제공된 자료에는 2018년 한국의 수출액에 대한 정보가 포함되어 있지 않습니다. 한...,[국제관광 출발 횟수100만 2038 2187 2394 263 2692\n국제관광 ...
8,한-인도 CEPA를 적용하여 수출하기 위해 필요한 서류는 무엇인가요?,한-인도 CEPA를 적용하여 수출하기 위해서는 원산지기준을 충족한다는 것을 증명하는...,한-인도 CEPA를 적용하여 수출하기 위해 필요한 주요 서류는 다음과 같습니다:\n...,[제Ⅱ절\n한-인도 CEPA 활용방안\n02\n한-인도 CEPA\n수출유망품목 및\...
9,인도 내 수입업자의 이름 및 주소는 필수 요구사항인가요?,국내 제조된 제품의 경우 인도 내 수입업자의 이름 및 주소는 생략 가능합니다.,제공된 자료에는 인도 내 수입업자의 이름 및 주소가 필수 요구사항인지에 대한 정보가...,[PostPaid후불결제의 경우 상기 서류 외에도 거주증명서와 같은 별도의 증빙서류...


In [6]:
len(qa_data)

10

In [7]:
from ragas import evaluate

result = evaluate(
    dataset,
    metrics=metrics,
    raise_exceptions=False  # Add this line
)

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

Failed to parse output. Returning None.


In [8]:
df1 = pd.DataFrame(list(result.items()), columns=['Metric', 'setting1'])


df1

,Metric,setting1
0,faithfulness,0.333333
1,answer_relevancy,0.589395
2,answer_correctness,0.352464
3,context_recall,0.200000
4,context_precision,0.691667


In [9]:
df1.set_index('Metric', inplace=True)
df1 = df1.transpose().reset_index()

df1

Metric,index,faithfulness,answer_relevancy,answer_correctness,context_recall,context_precision
0,setting1,0.333333,0.589395,0.352464,0.2,0.691667


In [10]:
import os 
import glob 

In [11]:
# 파일 경로 패턴 설정
file_pattern = './experiment_dataset/QA/setting1_*.xlsx'

# 파일 리스트 가져오기
file_list = glob.glob(file_pattern)

In [12]:
file_list

['./experiment_dataset/QA\\setting1_retriever1_003.xlsx',
 './experiment_dataset/QA\\setting1_retriever1_005.xlsx',
 './experiment_dataset/QA\\setting1_retriever1_007.xlsx',
 './experiment_dataset/QA\\setting1_retriever1_010.xlsx']

In [13]:
file_list[0].split('\\')[-1].split('.')[0]

'setting1_retriever1_003'

In [14]:
from ragas.run_config import RunConfig
# RAGAS Evaluation TimeoutError를 해결하기 위해 

In [15]:
def merge_result(file_list):
    for i, file in enumerate(file_list):
        set_name = file.split('\\')[-1].split('.')[0]
        print(set_name)
        _, dataset = set_data(file)
        # raise_exceptions=False로 하면 평가시 nan값이 발생했을 때, 평가 종합 계산시 제외한다.
        result = evaluate(dataset, 
                          metrics=metrics, 
                          raise_exceptions=False, 
                          run_config=RunConfig(timeout=120, max_retries=5, max_wait=120, max_workers=32)
                          )
        
        if i == 0:
            df = pd.DataFrame(list(result.items()) + [('mean', np.mean(list(result.values())))], columns=['Metric', set_name])

        else:
            df_temp = pd.DataFrame(list(result.items()) + [('mean', np.mean(list(result.values())))], columns=['Metric', set_name])
            df = pd.merge(df, df_temp, on='Metric')
        
        print(df)
    
    
    df.set_index('Metric', inplace=True)
    df = df.transpose().reset_index()
    
    return df
           

In [16]:
import nest_asyncio     
nest_asyncio.apply()  

In [17]:
df = merge_result(file_list)

setting1_retriever1_003


Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

Failed to parse output. Returning None.


               Metric  setting1_retriever1_003
0        faithfulness                 0.370370
1    answer_relevancy                 0.389986
2  answer_correctness                 0.332464
3      context_recall                 0.200000
4   context_precision                 0.733333
5                mean                 0.405231
setting1_retriever1_005


Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

...

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

               Metric  setting1_retriever1_003  setting1_retriever1_005  \
0        faithfulness                 0.370370                 0.373513   
1    answer_relevancy                 0.389986                 0.290081   
2  answer_correctness                 0.332464                 0.400448   
3      context_recall                 0.200000                 0.300000   
4   context_precision                 0.733333                 0.682222   
5                mean                 0.405231                 0.409253   

   setting1_retriever1_007  
0                 0.464113  
1                 0.387237  
2                 0.385723  
3                 0.300000  
4                 0.646230  
5                 0.436661  
setting1_retriever1_010


Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

Runner in Executor raised an exception
Traceback (most recent call last):
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\asyncio\tasks.py", line 520, in wait_for
    return await fut
           ^^^^^^^^^
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\ragas\metrics\_faithfulness.py", line 277, in _ascore
    await _faithfulness_output_parser.aparse(
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\ragas\llms\output_parser.py", line 67, in aparse
    output = await llm.generate(p_value)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\ragas\llms\base.py", line 93, in generate
    return await agenerate_text_with_retry(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\tenacity\asyncio\__init__.py", line 189, in async_wrapped
    return await copy(fn, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\kim_h\anaconda3

               Metric  setting1_retriever1_003  setting1_retriever1_005  \
0        faithfulness                 0.370370                 0.373513   
1    answer_relevancy                 0.389986                 0.290081   
2  answer_correctness                 0.332464                 0.400448   
3      context_recall                 0.200000                 0.300000   
4   context_precision                 0.733333                 0.682222   
5                mean                 0.405231                 0.409253   

   setting1_retriever1_007  setting1_retriever1_010  
0                 0.464113                 0.318519  
1                 0.387237                 0.287194  
2                 0.385723                 0.417108  
3                 0.300000                 0.400000  
4                 0.646230                 0.622044  
5                 0.436661                 0.408973  


In [18]:
df

Metric,index,faithfulness,answer_relevancy,answer_correctness,context_recall,context_precision,mean
0,setting1_retriever1_003,0.370370,0.389986,0.332464,0.2,0.733333,0.405231
1,setting1_retriever1_005,0.373513,0.290081,0.400448,0.3,0.682222,0.409253
2,setting1_retriever1_007,0.464113,0.387237,0.385723,0.3,0.646230,0.436661
3,setting1_retriever1_010,0.318519,0.287194,0.417108,0.4,0.622044,0.408973


In [19]:
df.set_index('Metric', inplace=True)
df = df.transpose().reset_index()

df

KeyError: "None of ['Metric'] are in the columns"

In [ ]:
df.to_excel('./experiment_dataset/QA_results/setting1_retriever1.xlsx', index=False)

In [ ]:
# 파일 경로 패턴 설정
file_pattern = './experiment_dataset/QA/setting2_*.xlsx'

# 파일 리스트 가져오기
file_list = glob.glob(file_pattern)

print(file_list)

['./experiment_dataset/QA\\setting2_retriever2_003.xlsx', './experiment_dataset/QA\\setting2_retriever2_005.xlsx', './experiment_dataset/QA\\setting2_retriever2_007.xlsx', './experiment_dataset/QA\\setting2_retriever2_010.xlsx']


In [ ]:
import nest_asyncio     
nest_asyncio.apply()  

In [ ]:
df = merge_result(file_list)

setting2_retriever2_003


Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

Runner in Executor raised an exception
Traceback (most recent call last):
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\asyncio\tasks.py", line 520, in wait_for
    return await fut
           ^^^^^^^^^
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\ragas\metrics\_faithfulness.py", line 277, in _ascore
    await _faithfulness_output_parser.aparse(
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\ragas\llms\output_parser.py", line 67, in aparse
    output = await llm.generate(p_value)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\ragas\llms\base.py", line 93, in generate
    return await agenerate_text_with_retry(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\tenacity\asyncio\__init__.py", line 189, in async_wrapped
    return await copy(fn, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\kim_h\anaconda3

               Metric  setting2_retriever2_003
0        faithfulness                 0.293561
1    answer_relevancy                 0.573299
2  answer_correctness                 0.354985
3      context_recall                 0.200000
4   context_precision                 0.733333
5                mean                 0.431036
setting2_retriever2_005


Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

...

Failed to parse output. Returning None.
Runner in Executor raised an exception
Traceback (most recent call last):
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\asyncio\tasks.py", line 520, in wait_for
    return await fut
           ^^^^^^^^^
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\ragas\metrics\_faithfulness.py", line 277, in _ascore
    await _faithfulness_output_parser.aparse(
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\ragas\llms\output_parser.py", line 67, in aparse
    output = await llm.generate(p_value)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\ragas\llms\base.py", line 93, in generate
    return await agenerate_text_with_retry(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\tenacity\asyncio\__init__.py", line 189, in async_wrapped
    return await copy(fn, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^

               Metric  setting2_retriever2_003  setting2_retriever2_005  \
0        faithfulness                 0.293561                 0.652422   
1    answer_relevancy                 0.573299                 0.583937   
2  answer_correctness                 0.354985                 0.450422   
3      context_recall                 0.200000                 0.300000   
4   context_precision                 0.733333                 0.687778   
5                mean                 0.431036                 0.534912   

   setting2_retriever2_007  
0                 0.524107  
1                 0.377467  
2                 0.363764  
3                 0.200000  
4                 0.562746  
5                 0.405617  
setting2_retriever2_010


Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

Failed to parse output. Returning None.
Runner in Executor raised an exception
Traceback (most recent call last):
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\asyncio\tasks.py", line 520, in wait_for
    return await fut
           ^^^^^^^^^
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\ragas\metrics\_faithfulness.py", line 277, in _ascore
    await _faithfulness_output_parser.aparse(
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\ragas\llms\output_parser.py", line 67, in aparse
    output = await llm.generate(p_value)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\ragas\llms\base.py", line 93, in generate
    return await agenerate_text_with_retry(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\tenacity\asyncio\__init__.py", line 189, in async_wrapped
    return await copy(fn, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^

               Metric  setting2_retriever2_003  setting2_retriever2_005  \
0        faithfulness                 0.293561                 0.652422   
1    answer_relevancy                 0.573299                 0.583937   
2  answer_correctness                 0.354985                 0.450422   
3      context_recall                 0.200000                 0.300000   
4   context_precision                 0.733333                 0.687778   
5                mean                 0.431036                 0.534912   

   setting2_retriever2_007  setting2_retriever2_010  
0                 0.524107                 0.656250  
1                 0.377467                 0.376830  
2                 0.363764                 0.394017  
3                 0.200000                 0.250000  
4                 0.562746                 0.595767  
5                 0.405617                 0.454573  


In [ ]:
df.set_index('Metric', inplace=True)
df = df.transpose().reset_index()

df

Metric,index,faithfulness,answer_relevancy,answer_correctness,context_recall,context_precision,mean
0,setting2_retriever2_003,0.293561,0.573299,0.354985,0.20,0.733333,0.431036
1,setting2_retriever2_005,0.652422,0.583937,0.450422,0.30,0.687778,0.534912
2,setting2_retriever2_007,0.524107,0.377467,0.363764,0.20,0.562746,0.405617
3,setting2_retriever2_010,0.656250,0.376830,0.394017,0.25,0.595767,0.454573


In [ ]:
df.to_excel('./experiment_dataset/QA_results/setting2_retriever2.xlsx', index=False)

In [ ]:
import nest_asyncio     
nest_asyncio.apply()  

# 파일 경로 패턴 설정
file_pattern = './experiment_dataset/QA/setting3_*.xlsx'

# 파일 리스트 가져오기
file_list = glob.glob(file_pattern)

print(file_list)

df = merge_result(file_list)

df.to_excel('./experiment_dataset/QA_results/setting3_retriever3.xlsx', index=False)

['./experiment_dataset/QA\\setting3_retriever3_003_001.xlsx', './experiment_dataset/QA\\setting3_retriever3_003_002.xlsx', './experiment_dataset/QA\\setting3_retriever3_003_003.xlsx', './experiment_dataset/QA\\setting3_retriever3_003_004.xlsx', './experiment_dataset/QA\\setting3_retriever3_003_005.xlsx', './experiment_dataset/QA\\setting3_retriever3_003_006.xlsx', './experiment_dataset/QA\\setting3_retriever3_003_007.xlsx', './experiment_dataset/QA\\setting3_retriever3_005_001.xlsx', './experiment_dataset/QA\\setting3_retriever3_005_002.xlsx', './experiment_dataset/QA\\setting3_retriever3_005_003.xlsx', './experiment_dataset/QA\\setting3_retriever3_005_004.xlsx', './experiment_dataset/QA\\setting3_retriever3_005_005.xlsx', './experiment_dataset/QA\\setting3_retriever3_005_006.xlsx', './experiment_dataset/QA\\setting3_retriever3_005_007.xlsx', './experiment_dataset/QA\\setting3_retriever3_007_001.xlsx', './experiment_dataset/QA\\setting3_retriever3_007_002.xlsx', './experiment_dataset/Q

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

               Metric  setting3_retriever3_003_001
0        faithfulness                     0.401010
1    answer_relevancy                     0.576248
2  answer_correctness                     0.368899
3      context_recall                     0.400000
4   context_precision                     0.550000
5                mean                     0.459231
setting3_retriever3_003_002


Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

Failed to parse output. Returning None.
Runner in Executor raised an exception
Traceback (most recent call last):
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\asyncio\tasks.py", line 520, in wait_for
    return await fut
           ^^^^^^^^^
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\ragas\metrics\_faithfulness.py", line 277, in _ascore
    await _faithfulness_output_parser.aparse(
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\ragas\llms\output_parser.py", line 67, in aparse
    output = await llm.generate(p_value)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\ragas\llms\base.py", line 93, in generate
    return await agenerate_text_with_retry(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\tenacity\asyncio\__init__.py", line 189, in async_wrapped
    return await copy(fn, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^

...

Failed to parse output. Returning None.
Runner in Executor raised an exception
Traceback (most recent call last):
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\asyncio\tasks.py", line 520, in wait_for
    return await fut
           ^^^^^^^^^
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\ragas\metrics\_faithfulness.py", line 277, in _ascore
    await _faithfulness_output_parser.aparse(
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\ragas\llms\output_parser.py", line 67, in aparse
    output = await llm.generate(p_value)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\ragas\llms\base.py", line 93, in generate
    return await agenerate_text_with_retry(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\tenacity\asyncio\__init__.py", line 189, in async_wrapped
    return await copy(fn, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^

               Metric  setting3_retriever3_003_001  \
0        faithfulness                     0.401010   
1    answer_relevancy                     0.576248   
2  answer_correctness                     0.368899   
3      context_recall                     0.400000   
4   context_precision                     0.550000   
5                mean                     0.459231   

   setting3_retriever3_003_002  setting3_retriever3_003_003  \
0                     0.430556                     0.614583   
1                     0.473512                     0.562394   
2                     0.418975                     0.451482   
3                     0.222222                     0.300000   
4                     0.575000                     0.533333   
5                     0.424053                     0.492359   

   setting3_retriever3_003_004  setting3_retriever3_003_005  \
0                     0.392857                     0.357639   
1                     0.570354                     0.

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

Failed to parse output. Returning None.
Failed to parse output. Returning None.
Runner in Executor raised an exception
Traceback (most recent call last):
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\asyncio\tasks.py", line 520, in wait_for
    return await fut
           ^^^^^^^^^
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\ragas\metrics\_faithfulness.py", line 277, in _ascore
    await _faithfulness_output_parser.aparse(
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\ragas\llms\output_parser.py", line 67, in aparse
    output = await llm.generate(p_value)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\ragas\llms\base.py", line 93, in generate
    return await agenerate_text_with_retry(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\tenacity\asyncio\__init__.py", line 189, in async_wrapped
    return await copy(fn, *args, **kwa

               Metric  setting3_retriever3_003_001  \
0        faithfulness                     0.401010   
1    answer_relevancy                     0.576248   
2  answer_correctness                     0.368899   
3      context_recall                     0.400000   
4   context_precision                     0.550000   
5                mean                     0.459231   

   setting3_retriever3_003_002  setting3_retriever3_003_003  \
0                     0.430556                     0.614583   
1                     0.473512                     0.562394   
2                     0.418975                     0.451482   
3                     0.222222                     0.300000   
4                     0.575000                     0.533333   
5                     0.424053                     0.492359   

   setting3_retriever3_003_004  setting3_retriever3_003_005  \
0                     0.392857                     0.357639   
1                     0.570354                     0.

In [ ]:
import nest_asyncio     
nest_asyncio.apply()  

# 파일 경로 패턴 설정
file_pattern = './experiment_dataset/QA/setting4_*.xlsx'

# 파일 리스트 가져오기
file_list = glob.glob(file_pattern)

print(file_list)

df = merge_result(file_list)

df.to_excel('./experiment_dataset/QA_results/setting4_our_retriever.xlsx', index=False)

['./experiment_dataset/QA\\setting4_our_retriever_003_001.xlsx', './experiment_dataset/QA\\setting4_our_retriever_003_002.xlsx', './experiment_dataset/QA\\setting4_our_retriever_003_003.xlsx', './experiment_dataset/QA\\setting4_our_retriever_003_004.xlsx', './experiment_dataset/QA\\setting4_our_retriever_003_005.xlsx', './experiment_dataset/QA\\setting4_our_retriever_003_006.xlsx', './experiment_dataset/QA\\setting4_our_retriever_003_007.xlsx', './experiment_dataset/QA\\setting4_our_retriever_005_001.xlsx', './experiment_dataset/QA\\setting4_our_retriever_005_002.xlsx', './experiment_dataset/QA\\setting4_our_retriever_005_003.xlsx', './experiment_dataset/QA\\setting4_our_retriever_005_004.xlsx', './experiment_dataset/QA\\setting4_our_retriever_005_005.xlsx', './experiment_dataset/QA\\setting4_our_retriever_005_006.xlsx', './experiment_dataset/QA\\setting4_our_retriever_005_007.xlsx', './experiment_dataset/QA\\setting4_our_retriever_007_001.xlsx', './experiment_dataset/QA\\setting4_our_

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

Runner in Executor raised an exception
Traceback (most recent call last):
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\asyncio\tasks.py", line 520, in wait_for
    return await fut
           ^^^^^^^^^
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\ragas\metrics\_faithfulness.py", line 277, in _ascore
    await _faithfulness_output_parser.aparse(
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\ragas\llms\output_parser.py", line 67, in aparse
    output = await llm.generate(p_value)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\ragas\llms\base.py", line 93, in generate
    return await agenerate_text_with_retry(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\tenacity\asyncio\__init__.py", line 189, in async_wrapped
    return await copy(fn, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\kim_h\anaconda3

               Metric  setting4_our_retriever_003_001
0        faithfulness                        0.437500
1    answer_relevancy                        0.670804
2  answer_correctness                        0.328672
3      context_recall                        0.200000
4   context_precision                        0.650000
5                mean                        0.457395
setting4_our_retriever_003_002


Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

...

Runner in Executor raised an exception
Traceback (most recent call last):
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\asyncio\tasks.py", line 520, in wait_for
    return await fut
           ^^^^^^^^^
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\ragas\metrics\_faithfulness.py", line 277, in _ascore
    await _faithfulness_output_parser.aparse(
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\ragas\llms\output_parser.py", line 67, in aparse
    output = await llm.generate(p_value)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\ragas\llms\base.py", line 93, in generate
    return await agenerate_text_with_retry(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\tenacity\asyncio\__init__.py", line 189, in async_wrapped
    return await copy(fn, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\kim_h\anaconda3

               Metric  setting4_our_retriever_003_001  \
0        faithfulness                        0.437500   
1    answer_relevancy                        0.670804   
2  answer_correctness                        0.328672   
3      context_recall                        0.200000   
4   context_precision                        0.650000   
5                mean                        0.457395   

   setting4_our_retriever_003_002  setting4_our_retriever_003_003  \
0                        0.292361                        0.236111   
1                        0.470568                        0.663371   
2                        0.341553                        0.387753   
3                        0.200000                        0.200000   
4                        0.600000                        0.708333   
5                        0.380896                        0.439114   

   setting4_our_retriever_003_004  setting4_our_retriever_003_005  \
0                        0.420940              

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

Runner in Executor raised an exception
Traceback (most recent call last):
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\asyncio\tasks.py", line 520, in wait_for
    return await fut
           ^^^^^^^^^
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\ragas\metrics\_faithfulness.py", line 277, in _ascore
    await _faithfulness_output_parser.aparse(
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\ragas\llms\output_parser.py", line 67, in aparse
    output = await llm.generate(p_value)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\ragas\llms\base.py", line 93, in generate
    return await agenerate_text_with_retry(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\tenacity\asyncio\__init__.py", line 189, in async_wrapped
    return await copy(fn, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\kim_h\anaconda3

               Metric  setting4_our_retriever_003_001  \
0        faithfulness                        0.437500   
1    answer_relevancy                        0.670804   
2  answer_correctness                        0.328672   
3      context_recall                        0.200000   
4   context_precision                        0.650000   
5                mean                        0.457395   

   setting4_our_retriever_003_002  setting4_our_retriever_003_003  \
0                        0.292361                        0.236111   
1                        0.470568                        0.663371   
2                        0.341553                        0.387753   
3                        0.200000                        0.200000   
4                        0.600000                        0.708333   
5                        0.380896                        0.439114   

   setting4_our_retriever_003_004  setting4_our_retriever_003_005  \
0                        0.420940              

In [ ]:
import nest_asyncio     
nest_asyncio.apply()  

# 파일 경로 패턴 설정
file_pattern = './experiment_dataset/QA/setting5_*.xlsx'

# 파일 리스트 가져오기
file_list = glob.glob(file_pattern)

print(file_list)

df = merge_result(file_list)

df.to_excel('./experiment_dataset/QA_results/setting5_mq_retriever.xlsx', index=False)



['./experiment_dataset/QA\\setting5_mq_retriever.xlsx']
setting5_mq_retriever


Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

Failed to parse output. Returning None.


               Metric  setting5_mq_retriever
0        faithfulness               0.256614
1    answer_relevancy               0.277711
2  answer_correctness               0.413804
3      context_recall               0.300000
4   context_precision               0.670099
5                mean               0.383646


Metric,index,faithfulness,answer_relevancy,answer_correctness,context_recall,context_precision,mean
0,setting5_mq_retriever,0.256614,0.277711,0.413804,0.3,0.670099,0.383646


In [ ]:
df.head()

Metric,index,faithfulness,answer_relevancy,answer_correctness,context_recall,context_precision,mean
0,setting6_sq_retriever,0.290741,0.679777,0.383608,0.111111,0.458333,0.384714


In [ ]:
import nest_asyncio     
nest_asyncio.apply()  

# 파일 경로 패턴 설정
file_pattern = './experiment_dataset/QA/setting6_*.xlsx'

# 파일 리스트 가져오기
file_list = glob.glob(file_pattern)

print(file_list)

df = merge_result(file_list)

df.to_excel('./experiment_dataset/QA_results/setting6_sq_retriever.xlsx', index=False)


['./experiment_dataset/QA\\setting6_sq_retriever.xlsx']
setting6_sq_retriever


Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

Failed to parse output. Returning None.
Runner in Executor raised an exception
Traceback (most recent call last):
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\asyncio\tasks.py", line 520, in wait_for
    return await fut
           ^^^^^^^^^
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\ragas\metrics\_faithfulness.py", line 277, in _ascore
    await _faithfulness_output_parser.aparse(
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\ragas\llms\output_parser.py", line 67, in aparse
    output = await llm.generate(p_value)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\ragas\llms\base.py", line 93, in generate
    return await agenerate_text_with_retry(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\tenacity\asyncio\__init__.py", line 189, in async_wrapped
    return await copy(fn, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^

               Metric  setting6_sq_retriever
0        faithfulness               0.290741
1    answer_relevancy               0.679777
2  answer_correctness               0.383608
3      context_recall               0.111111
4   context_precision               0.458333
5                mean               0.384714


In [ ]:

df.head()

Metric,index,faithfulness,answer_relevancy,answer_correctness,context_recall,context_precision,mean
0,setting6_sq_retriever,0.290741,0.679777,0.383608,0.111111,0.458333,0.384714
